In [2]:
from itertools import chain

# import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import RandomizedSearchCV

import numpy as np
from sklearn.model_selection import KFold
from utils_2 import *
import pickle
import time
from sklearn.ensemble import RandomForestClassifier

from sklearn.externals import joblib
from sklearn.model_selection import GridSearchCV

from sklearn.cross_validation import train_test_split

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.callbacks import LearningRateScheduler
from keras.callbacks import Callback

/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
Using TensorF

In [3]:
all_data, all_files = read_training_data_raw(training_path = '/home/tranvanluan7/sepsis_detection/new_training/')
mean_features = [data.mean(axis=0, skipna=True) for data in all_data]
concate_features = pd.concat(mean_features, axis=1)
mean_features = concate_features.mean(axis=1, skipna=True)
print(mean_features)


# impute missing data
def impute_missing_data(df):
    #df = df.interpolate(method ='linear')
    df2 = df.copy()
    for column_name in df2.columns:
        replace_value = mean_features[column_name]
        df2[column_name] = df2[column_name].fillna(value=replace_value)
    return df2

imputed_data = []
for idx,data in enumerate(all_data):
    imputed_data.append(impute_missing_data(data))
    if idx % 1000 == 0:
        print("finished imputing, ",idx)
        
all_data_imputed_values = [x.loc[:,x.columns!='SepsisLabel'] for x in imputed_data]
all_data_labels = [x.loc[:, x.columns=='SepsisLabel'] for x in all_data]
all_data_raw_values =  [x.loc[:,x.columns!='SepsisLabel'] for x in all_data]



('Finished ', 0)
('Finished ', 500)
('Finished ', 1000)
('Finished ', 1500)
('Finished ', 2000)
('Finished ', 2500)
('Finished ', 3000)
('Finished ', 3500)
('Finished ', 4000)
('Finished ', 4500)
('Finished ', 5000)
('Finished ', 5500)
('Finished ', 6000)
('Finished ', 6500)
('Finished ', 7000)
('Finished ', 7500)
('Finished ', 8000)
('Finished ', 8500)
('Finished ', 9000)
('Finished ', 9500)
('Finished ', 10000)
('Finished ', 10500)
('Finished ', 11000)
('Finished ', 11500)
('Finished ', 12000)
('Finished ', 12500)
('Finished ', 13000)
('Finished ', 13500)
('Finished ', 14000)
('Finished ', 14500)
('Finished ', 15000)
('Finished ', 15500)
('Finished ', 16000)
('Finished ', 16500)
('Finished ', 17000)
('Finished ', 17500)
('Finished ', 18000)
('Finished ', 18500)
('Finished ', 19000)
('Finished ', 19500)
('Finished ', 20000)
('Finished ', 20500)
('Finished ', 21000)
('Finished ', 21500)
('Finished ', 22000)
('Finished ', 22500)
('Finished ', 23000)
('Finished ', 23500)
('Finished ', 24

In [4]:
#Separate into train,  test data
X_imputed_train, X_imputed_test, y_imputed_train, y_imputed_test, X_raw_train, X_raw_test = train_test_split(
    all_data_imputed_values, all_data_labels,
    all_data_raw_values, 
    test_size=0.33, random_state=42)

In [5]:
#Compute features
print("Computing features for net ")
nr_imputed_features = []
nr_labels = []
train_labels = []
for idx, data in enumerate(X_imputed_train):
    num_row = data.shape[0]
    num_col = data.shape[1]
    
    for i in range(5, num_row-1):
        d = data.loc[(i-5):i].values #Select recent 6 hours data as features 
        
        missing_percent = X_raw_train[idx].loc[(i-5):i].isna().mean().round(4).sum()
        
        l = int(y_imputed_train[idx].loc[i])
        if np.random.randint(0,100) > 95 or l == 1: #Select only 25% label 0
            nr_imputed_features.append(d)
            train_labels.append(l)
            if l == 1:
                nr_labels.append([0,1])
            else:
                nr_labels.append([1,0])
    if idx % 1000 == 0:
        print("Finished computing features", idx)

#Reshap train features for normalization
train_features = np.array(nr_imputed_features)
train_features = train_features.reshape((-1, 240))
print("Train feature shape", train_features.shape)

scaler = MinMaxScaler()
normalized_train_features = scaler.fit_transform(train_features)
normalized_train_features = normalized_train_features.reshape((-1, 6, 40))

print("After normalized, shape = ", normalized_train_features.shape)

Computing features for net 
('Finished computing features', 0)
('Finished computing features', 1000)
('Finished computing features', 2000)
('Finished computing features', 3000)
('Finished computing features', 4000)
('Finished computing features', 5000)
('Finished computing features', 6000)
('Finished computing features', 7000)
('Finished computing features', 8000)
('Finished computing features', 9000)
('Finished computing features', 10000)
('Finished computing features', 11000)
('Finished computing features', 12000)
('Finished computing features', 13000)
('Finished computing features', 14000)
('Finished computing features', 15000)
('Finished computing features', 16000)
('Finished computing features', 17000)
('Finished computing features', 18000)
('Finished computing features', 19000)
('Finished computing features', 20000)
('Finished computing features', 21000)
('Finished computing features', 22000)
('Finished computing features', 23000)
('Finished computing features', 24000)
('Finished

In [6]:
from keras.layers import *
from keras.models import Model
# LSTM Autoencoder model
# this is the size of our encoded representations
encoding_dim = 64  

# this is our input placeholder
input_img = Input(shape=(6, 40))
x = BatchNormalization()(input_img)
encoded = LSTM(128, activation='relu', return_sequences=True)(x)
encoded = LSTM(64, activation='relu', return_sequences=True)(encoded)
encoded = LSTM(encoding_dim, activation='relu')(encoded)

decoded = RepeatVector(6)(encoded)
decoded = LSTM(64, activation='relu', return_sequences=True)(decoded)
decoded = LSTM(128, activation='relu', return_sequences=True)(decoded)
decoded = TimeDistributed(Dense(40, activation='sigmoid'))(decoded)

classification_out = Dense(32, activation='relu')(encoded)
classification_out = Dropout(0.3)(classification_out)
classification_out = Dense(24, activation='relu')(classification_out)
classification_out = Dropout(0.3)(classification_out)
classification_out = Dense(16, activation='relu')(classification_out)
classification_out = Dropout(0.3)(classification_out)
classification_out = Dense(8, activation='relu')(classification_out)
classification_out = Dense(2, activation='softmax')(classification_out)
# this model maps an input to its reconstruction
autoencoder = Model(input_img, [decoded, classification_out])
encoder = Model(input_img, encoded)
classifier = Model(input_img, classification_out)


from keras.optimizers import Adam
from keras.optimizers import SGD
adam = Adam(lr=0.001)
autoencoder.compile(optimizer=adam, loss=['mean_squared_error','categorical_crossentropy'],
                    loss_weights=[0.2, 0.8])



Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
#Train model
filepath = 'best_model_autoencoder_Manh_RF'
    
checkpoint = ModelCheckpoint(
    filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

early_stop = EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=100, verbose=0, mode='auto')

sd=[]
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = [1,1]

    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        sd.append(step_decay(len(self.losses)))
        print('lr:', step_decay(len(self.losses)))

def step_decay(losses):
    if history.losses[-1]  < 0.48:
        lrate=0.001*1/(1+0.01*len(history.losses))
        momentum=0.9
        decay_rate=2e-6
    else:
        lrate = 0.001
    return lrate

history=LossHistory()
lrate=LearningRateScheduler(step_decay)

callbacks_list = [checkpoint, early_stop, history, lrate]
autoencoder.fit(normalized_train_features, [normalized_train_features, np.array(nr_labels)],
                epochs=100,
                batch_size=256,callbacks=callbacks_list,
                shuffle=True, validation_split=0.2)


Instructions for updating:
Use tf.cast instead.
Train on 39633 samples, validate on 9909 samples
Epoch 1/100
39633/39633 [==============================] - 36s 903us/step - loss: 0.4641 - time_distributed_1_loss: 0.0486 - dense_6_loss: 0.5680 - val_loss: 0.4197 - val_time_distributed_1_loss: 0.0171 - val_dense_6_loss: 0.5203

Epoch 00001: val_loss improved from inf to 0.41966, saving model to best_model_autoencoder_Manh_RF
('lr:', 0.000970873786407767)
Epoch 2/100
39633/39633 [==============================] - 10s 256us/step - loss: 0.4042 - time_distributed_1_loss: 0.0159 - dense_6_loss: 0.5012 - val_loss: 0.3930 - val_time_distributed_1_loss: 0.0155 - val_dense_6_loss: 0.4873

Epoch 00002: val_loss improved from 0.41966 to 0.39298, saving model to best_model_autoencoder_Manh_RF
('lr:', 0.0009615384615384615)
Epoch 3/100
39633/39633 [==============================] - 11s 267us/step - loss: 0.3804 - time_distributed_1_loss: 0.0137 - dense_6_loss: 0.4721 - val_loss: 0.3876 - val_time_di


Epoch 00025: val_loss did not improve from 0.38759
('lr:', 0.0007874015748031496)
Epoch 26/100
39633/39633 [==============================] - 12s 299us/step - loss: 0.1237 - time_distributed_1_loss: 0.0037 - dense_6_loss: 0.1537 - val_loss: 1.0765 - val_time_distributed_1_loss: 0.0041 - val_dense_6_loss: 1.3446

Epoch 00026: val_loss did not improve from 0.38759
('lr:', 0.00078125)
Epoch 27/100
39633/39633 [==============================] - 10s 255us/step - loss: 0.1157 - time_distributed_1_loss: 0.0039 - dense_6_loss: 0.1437 - val_loss: 0.9934 - val_time_distributed_1_loss: 0.0039 - val_dense_6_loss: 1.2407

Epoch 00027: val_loss did not improve from 0.38759
('lr:', 0.0007751937984496124)
Epoch 28/100
39633/39633 [==============================] - 10s 255us/step - loss: 0.1069 - time_distributed_1_loss: 0.0038 - dense_6_loss: 0.1327 - val_loss: 1.0385 - val_time_distributed_1_loss: 0.0039 - val_dense_6_loss: 1.2971

Epoch 00028: val_loss did not improve from 0.38759
('lr:', 0.0007692

39633/39633 [==============================] - 10s 260us/step - loss: 0.0383 - time_distributed_1_loss: 0.0037 - dense_6_loss: 0.0469 - val_loss: 1.5470 - val_time_distributed_1_loss: 0.0039 - val_dense_6_loss: 1.9328

Epoch 00052: val_loss did not improve from 0.38759
('lr:', 0.0006493506493506494)
Epoch 53/100
39633/39633 [==============================] - 10s 260us/step - loss: 0.0365 - time_distributed_1_loss: 0.0036 - dense_6_loss: 0.0447 - val_loss: 1.6867 - val_time_distributed_1_loss: 0.0039 - val_dense_6_loss: 2.1074

Epoch 00053: val_loss did not improve from 0.38759
('lr:', 0.0006451612903225806)
Epoch 54/100
39633/39633 [==============================] - 10s 258us/step - loss: 0.0350 - time_distributed_1_loss: 0.0036 - dense_6_loss: 0.0429 - val_loss: 1.6699 - val_time_distributed_1_loss: 0.0039 - val_dense_6_loss: 2.0864

Epoch 00054: val_loss did not improve from 0.38759
('lr:', 0.000641025641025641)
Epoch 55/100
39633/39633 [==============================] - 10s 250us/st

39633/39633 [==============================] - 10s 256us/step - loss: 0.0197 - time_distributed_1_loss: 0.0034 - dense_6_loss: 0.0238 - val_loss: 1.7770 - val_time_distributed_1_loss: 0.0038 - val_dense_6_loss: 2.2203

Epoch 00078: val_loss did not improve from 0.38759
('lr:', 0.0005555555555555556)
Epoch 79/100
39633/39633 [==============================] - 10s 261us/step - loss: 0.0195 - time_distributed_1_loss: 0.0034 - dense_6_loss: 0.0235 - val_loss: 1.7415 - val_time_distributed_1_loss: 0.0037 - val_dense_6_loss: 2.1759

Epoch 00079: val_loss did not improve from 0.38759
('lr:', 0.0005524861878453039)
Epoch 80/100
39633/39633 [==============================] - 10s 257us/step - loss: 0.0199 - time_distributed_1_loss: 0.0033 - dense_6_loss: 0.0241 - val_loss: 1.8233 - val_time_distributed_1_loss: 0.0037 - val_dense_6_loss: 2.2782

Epoch 00080: val_loss did not improve from 0.38759
('lr:', 0.0005494505494505495)
Epoch 81/100
39633/39633 [==============================] - 10s 252us/s

In [8]:
#Predict test data
autoencoder.load_weights(filepath)
#transformed_features = encoder.predict(normalized_train_features)

# test autoencoder + nr classifier model with test data
cohort_labels = []
cohort_predictions = []
cohort_probabilities = []
for idx, data in enumerate(X_imputed_test):
    num_row = data.shape[0]
    num_col = data.shape[1]
    x_input = []
    labels = []
    for i in range(num_row):
        if i >=5:
            d = data.loc[(i-5):i].values
        else:
            imputing_values = np.array(mean_features.values[0:40]).reshape(1, 40)
            d = data.loc[0:i].values
            
            while(d.shape[0] < 6):
                d = np.concatenate([imputing_values, d], axis=0)
            
        d = d.reshape((6, 40)).flatten()
        l = int(y_imputed_test[idx].loc[i])
        labels.append(l)
        x_input.append(d)
        
        
    x_input = np.array(x_input)
    
    nr_output = classifier.predict(scaler.transform(x_input).reshape((len(x_input), 6, 40)))
    probabilities = nr_output[:,1]
    predictions = []
    for p in probabilities:
        if p > 0.5:
            predictions.append(1)
        else: predictions.append(0)
            
    predictions = np.array(predictions)
    
    cohort_labels.append(labels)
    cohort_predictions.append(predictions)
    cohort_probabilities.append(probabilities)
    if idx % 1000 == 0:
        print("Finished predicting", idx)


('Finished predicting', 0)
('Finished predicting', 1000)
('Finished predicting', 2000)
('Finished predicting', 3000)
('Finished predicting', 4000)
('Finished predicting', 5000)
('Finished predicting', 6000)
('Finished predicting', 7000)
('Finished predicting', 8000)
('Finished predicting', 9000)
('Finished predicting', 10000)
('Finished predicting', 11000)
('Finished predicting', 12000)
('Finished predicting', 13000)


In [9]:
#Compute utility
def compute_auc(labels, predictions):
    # Check inputs for errors.
    if len(predictions) != len(labels):
        raise Exception('Numbers of predictions and labels must be the same.')

    n = len(labels)
    for i in range(n):
        if not labels[i] in (0, 1):
            raise Exception('Labels must satisfy label == 0 or label == 1.')

    for i in range(n):
        if not 0 <= predictions[i] <= 1:
            raise Exception('Predictions must satisfy 0 <= prediction <= 1.')

    # Find prediction thresholds.
    thresholds = np.unique(predictions)[::-1]
    if thresholds[0] != 1:
        thresholds = np.concatenate((np.array([1]), thresholds))

    if thresholds[-1] != 0:
        thresholds = np.concatenate((thresholds, np.array([0])))
    m = len(thresholds)

    # Populate contingency table across prediction thresholds.
    tp = np.zeros(m)
    fp = np.zeros(m)
    fn = np.zeros(m)
    tn = np.zeros(m)

    # Find indices that sort predicted probabilities from largest to smallest.
    idx = np.argsort(predictions)[::-1]

    i = 0
    for j in range(m):
        # Initialize contingency table for j-th prediction threshold.
        if j == 0:
            tp[j] = 0
            fp[j] = 0
            fn[j] = np.sum(labels == 1)
            tn[j] = np.sum(labels == 0)
        else:
            tp[j] = tp[j - 1]
            fp[j] = fp[j - 1]
            fn[j] = fn[j - 1]
            tn[j] = tn[j - 1]

        # Update contingency table for i-th largest prediction probability.
        while i < n and predictions[idx[i]] >= thresholds[j]:
            if labels[idx[i]]:
                tp[j] += 1
                fn[j] -= 1
            else:
                fp[j] += 1
                tn[j] -= 1
            i += 1

    # Summarize contingency table.
    tpr = np.zeros(m)
    tnr = np.zeros(m)
    ppv = np.zeros(m)
    npv = np.zeros(m)

    for j in range(m):
        if tp[j] + fn[j]:
            tpr[j] = tp[j] / (tp[j] + fn[j])
        else:
            tpr[j] = 1
        if fp[j] + tn[j]:
            tnr[j] = tn[j] / (fp[j] + tn[j])
        else:
            tnr[j] = 1
        if tp[j] + fp[j]:
            ppv[j] = tp[j] / (tp[j] + fp[j])
        else:
            ppv[j] = 1
        if fn[j] + tn[j]:
            npv[j] = tn[j] / (fn[j] + tn[j])
        else:
            npv[j] = 1

    # Compute AUROC as the area under a piecewise linear function of TPR /
    # sensitivity (x-axis) and TNR / specificity (y-axis) and AUPRC as the area
    # under a piecewise constant of TPR / recall (x-axis) and PPV / precision
    # (y-axis).
    auroc = 0
    auprc = 0
    for j in range(m-1):
        auroc += 0.5 * (tpr[j + 1] - tpr[j]) * (tnr[j + 1] + tnr[j])
        auprc += (tpr[j + 1] - tpr[j]) * ppv[j + 1]

    return auroc, auprc
def compute_accuracy_f_measure(labels, predictions):
    # Check inputs for errors.
    if len(predictions) != len(labels):
        raise Exception('Numbers of predictions and labels must be the same.')

    n = len(labels)
    for i in range(n):
        if not labels[i] in (0, 1):
            raise Exception('Labels must satisfy label == 0 or label == 1.')

    for i in range(n):
        if not predictions[i] in (0, 1):
            raise Exception(
                'Predictions must satisfy prediction == 0 or prediction == 1.')

    # Populate contingency table.
    tp = 0
    fp = 0
    fn = 0
    tn = 0

    for i in range(n):
        if labels[i] and predictions[i]:
            tp += 1
        elif labels[i] and not predictions[i]:
            fp += 1
        elif not labels[i] and predictions[i]:
            fn += 1
        elif not labels[i] and not predictions[i]:
            tn += 1

    # Summarize contingency table.
    if tp + fp + fn + tn:
        accuracy = float(tp + tn) / float(tp + fp + fn + tn)
    else:
        accuracy = 1.0

    if 2 * tp + fp + fn:
        f_measure = float(2 * tp) / float(2 * tp + fp + fn)
    else:
        f_measure = 1.0

    return accuracy, f_measure

def compute_prediction_utility(labels, predictions, dt_early=-12, dt_optimal=-6, dt_late=3.0, max_u_tp=1, min_u_fn=-2, u_fp=-0.05, u_tn=0):
    # Check inputs for errors.
    if len(predictions) != len(labels):
        raise Exception('Numbers of predictions and labels must be the same.')

    n = len(labels)
    for i in range(n):
        if not labels[i] in (0, 1):
            raise Exception('Labels must satisfy label == 0 or label == 1.')

    for i in range(n):
        if not predictions[i] in (0, 1):
            raise Exception(
                'Predictions must satisfy prediction == 0 or prediction == 1.')

    if dt_early >= dt_optimal:
        raise Exception(
            'The earliest beneficial time for predictions must be before the optimal time.')

    if dt_optimal >= dt_late:
        raise Exception(
            'The optimal time for predictions must be before the latest beneficial time.')

    # Does the patient eventually have sepsis?
    if any(labels):
        is_septic = True
        t_sepsis = min(i for i, label in enumerate(
            labels) if label) - dt_optimal
    else:
        is_septic = False
        t_sepsis = float('inf')

    # Define slopes and intercept points for affine utility functions of the
    # form u = m * t + b.
    m_1 = float(max_u_tp) / float(dt_optimal - dt_early)
    b_1 = -m_1 * dt_early
    m_2 = float(-max_u_tp) / float(dt_late - dt_optimal)
    b_2 = -m_2 * dt_late
    m_3 = float(min_u_fn) / float(dt_late - dt_optimal)
    b_3 = -m_3 * dt_optimal

    # Compare predicted and true conditions.
    u = np.zeros(n)
    for t in range(n):
        if t <= t_sepsis + dt_late:
            # TP
            if is_septic and predictions[t]:
                if t <= t_sepsis + dt_optimal:
                    u[t] = max(m_1 * (t - t_sepsis) + b_1, u_fp)
                elif t <= t_sepsis + dt_late:
                    u[t] = m_2 * (t - t_sepsis) + b_2
            # FN
            elif is_septic and not predictions[t]:
                if t <= t_sepsis + dt_optimal:
                    u[t] = 0
                elif t <= t_sepsis + dt_late:
                    u[t] = m_3 * (t - t_sepsis) + b_3
            # FP
            elif not is_septic and predictions[t]:
                u[t] = u_fp
            # TN
            elif not is_septic and not predictions[t]:
                u[t] = u_tn

    # Find total utility for patient.
    return np.sum(u)


In [10]:

labels = np.concatenate(cohort_labels)
predictions = np.concatenate(cohort_predictions)
probabilities = np.concatenate(cohort_probabilities)

auroc, auprc = compute_auc(labels, probabilities)
accuracy, f_measure = compute_accuracy_f_measure(labels, predictions)

print("AUCROC, AUCPRC")
print(auroc, auprc)
print("Accuaracy, F-measure")
print(accuracy, f_measure) 

num_files = len(y_imputed_test)
dt_early = -12
dt_optimal = -6
dt_late = 3

max_u_tp = 1
min_u_fn = -2
u_fp = -0.05
u_tn = 0
# Compute utility.
observed_utilities = np.zeros(num_files)
best_utilities = np.zeros(num_files)
worst_utilities = np.zeros(num_files)
inaction_utilities = np.zeros(num_files)

for k in range(num_files):
    labels = cohort_labels[k]
    num_records = len(labels)
    observed_predictions = cohort_predictions[k]
    best_predictions = np.zeros(num_records)
    worst_predictions = np.zeros(num_records)
    inaction_predictions = np.zeros(num_records)

    if any(labels):
        t_sepsis = min(i for i, label in enumerate(
            labels) if label) - dt_optimal
        best_predictions[max(0, t_sepsis + dt_early)
                             : min(t_sepsis + dt_late, num_records)] = 1
    else:
        best_predictions[:] = 0
    worst_predictions = 1 - best_predictions

    observed_utilities[k] = compute_prediction_utility(
        labels, observed_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)
    best_utilities[k] = compute_prediction_utility(
        labels, best_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)
    worst_utilities[k] = compute_prediction_utility(
        labels, worst_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)
    inaction_utilities[k] = compute_prediction_utility(
        labels, inaction_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)

unnormalized_observed_utility = np.sum(observed_utilities)
unnormalized_best_utility = np.sum(best_utilities)
unnormalized_worst_utility = np.sum(worst_utilities)
unnormalized_inaction_utility = np.sum(inaction_utilities)

if not (unnormalized_worst_utility <= unnormalized_best_utility and unnormalized_inaction_utility <= unnormalized_best_utility):
    raise Exception(
        'Optimal utility must be higher than inaction utility.')

normalized_observed_utility = (unnormalized_observed_utility - unnormalized_inaction_utility) / (
    unnormalized_best_utility - unnormalized_inaction_utility)

print("Utility, ", normalized_observed_utility)

AUCROC, AUCPRC
(0.7944489676226631, 0.08336374148205733)
Accuaracy, F-measure
(0.8847423624163688, 0.1330813519096383)
('Utility, ', 0.3693794719279285)


In [11]:
# Test random forest
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=500, max_depth=70,random_state=0, bootstrap=True, max_features='auto',
                             min_samples_leaf=4, min_samples_split=10)

In [12]:
rf_features = normalized_train_features.reshape(-1, 240)
count0 = 0
count1 = 0
train_labels = []
for y in nr_labels:
    if y[0] == 0:
        train_labels.append(1)
        count1 +=1
    else:
        train_labels.append(0)
        count0 +=1
clf.fit(rf_features, train_labels)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=70, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [13]:
# test RF  model with test data
cohort_labels = []
cohort_predictions = []
cohort_probabilities = []
for idx, data in enumerate(X_imputed_test):
    num_row = data.shape[0]
    num_col = data.shape[1]
    x_input = []
    labels = []
    for i in range(num_row):
        if i >=5:
            d = data.loc[(i-5):i].values
        else:
            imputing_values = np.array(mean_features.values[0:40]).reshape(1, 40)
            d = data.loc[0:i].values
            
            while(d.shape[0] < 6):
                d = np.concatenate([imputing_values, d], axis=0)
            
        d = d.reshape((6, 40)).flatten()
        l = int(y_imputed_test[idx].loc[i])
        labels.append(l)
        x_input.append(d)
        
        
    x_input = np.array(x_input)
    
    nr_output = clf.predict_proba(scaler.transform(x_input))
    probabilities = nr_output[:,1]
    predictions = []
    for p in probabilities:
        if p > 0.5:
            predictions.append(1)
        else: predictions.append(0)
            
    predictions = np.array(predictions)
    
    cohort_labels.append(labels)
    cohort_predictions.append(predictions)
    cohort_probabilities.append(probabilities)
    if idx % 1000 == 0:
        print("Finished predicting", idx)


('Finished predicting', 0)
('Finished predicting', 1000)
('Finished predicting', 2000)
('Finished predicting', 3000)
('Finished predicting', 4000)
('Finished predicting', 5000)
('Finished predicting', 6000)
('Finished predicting', 7000)
('Finished predicting', 8000)
('Finished predicting', 9000)
('Finished predicting', 10000)
('Finished predicting', 11000)
('Finished predicting', 12000)
('Finished predicting', 13000)


In [14]:
#compute utility for random forest 
labels = np.concatenate(cohort_labels)
predictions = np.concatenate(cohort_predictions)
probabilities = np.concatenate(cohort_probabilities)

auroc, auprc = compute_auc(labels, probabilities)
accuracy, f_measure = compute_accuracy_f_measure(labels, predictions)

print("AUCROC, AUCPRC")
print(auroc, auprc)
print("Accuaracy, F-measure")
print(accuracy, f_measure) 

num_files = len(y_imputed_test)
dt_early = -12
dt_optimal = -6
dt_late = 3

max_u_tp = 1
min_u_fn = -2
u_fp = -0.05
u_tn = 0
# Compute utility.
observed_utilities = np.zeros(num_files)
best_utilities = np.zeros(num_files)
worst_utilities = np.zeros(num_files)
inaction_utilities = np.zeros(num_files)

for k in range(num_files):
    labels = cohort_labels[k]
    num_records = len(labels)
    observed_predictions = cohort_predictions[k]
    best_predictions = np.zeros(num_records)
    worst_predictions = np.zeros(num_records)
    inaction_predictions = np.zeros(num_records)

    if any(labels):
        t_sepsis = min(i for i, label in enumerate(
            labels) if label) - dt_optimal
        best_predictions[max(0, t_sepsis + dt_early)
                             : min(t_sepsis + dt_late, num_records)] = 1
    else:
        best_predictions[:] = 0
    worst_predictions = 1 - best_predictions

    observed_utilities[k] = compute_prediction_utility(
        labels, observed_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)
    best_utilities[k] = compute_prediction_utility(
        labels, best_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)
    worst_utilities[k] = compute_prediction_utility(
        labels, worst_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)
    inaction_utilities[k] = compute_prediction_utility(
        labels, inaction_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)

unnormalized_observed_utility = np.sum(observed_utilities)
unnormalized_best_utility = np.sum(best_utilities)
unnormalized_worst_utility = np.sum(worst_utilities)
unnormalized_inaction_utility = np.sum(inaction_utilities)

if not (unnormalized_worst_utility <= unnormalized_best_utility and unnormalized_inaction_utility <= unnormalized_best_utility):
    raise Exception(
        'Optimal utility must be higher than inaction utility.')

normalized_observed_utility = (unnormalized_observed_utility - unnormalized_inaction_utility) / (
    unnormalized_best_utility - unnormalized_inaction_utility)

print("Utility, ", normalized_observed_utility)

AUCROC, AUCPRC
(0.8170527206758389, 0.0906371017130265)
Accuaracy, F-measure
(0.9261591500053563, 0.16386934562537217)
('Utility, ', 0.3472239533617853)


In [15]:
#Save rf model
from sklearn.externals import joblib
rf_filepath = 'best_model_rf_FT'
joblib.dump(clf, rf_filepath, compress=True)

['best_model_rf_FT']

In [16]:
#XGBoost
from xgboost import XGBClassifier
xgb = XGBClassifier(objective= 'binary:logistic', colsample_bytree=0.8, learning_rate=0.1, min_child_weight=5,
                    n_estimators=500, subsample=0.8, max_depth=5, gamma=0.1)



In [17]:
xgb.fit(rf_features, train_labels)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.1, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=5, missing=None,
       n_estimators=500, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=0.8)

In [18]:
# test XGboost  model with test data
cohort_labels = []
cohort_predictions = []
cohort_probabilities = []
for idx, data in enumerate(X_imputed_test):
    num_row = data.shape[0]
    num_col = data.shape[1]
    x_input = []
    labels = []
    for i in range(num_row):
        if i >=5:
            d = data.loc[(i-5):i].values
        else:
            imputing_values = np.array(mean_features.values[0:40]).reshape(1, 40)
            d = data.loc[0:i].values
            
            while(d.shape[0] < 6):
                d = np.concatenate([imputing_values, d], axis=0)
            
        d = d.reshape((6, 40)).flatten()
        l = int(y_imputed_test[idx].loc[i])
        labels.append(l)
        x_input.append(d)
        
        
    x_input = np.array(x_input)
    
    nr_output = xgb.predict_proba(scaler.transform(x_input))
    probabilities = nr_output[:,1]
    predictions = []
    for p in probabilities:
        if p > 0.5:
            predictions.append(1)
        else: predictions.append(0)
            
    predictions = np.array(predictions)
    
    cohort_labels.append(labels)
    cohort_predictions.append(predictions)
    cohort_probabilities.append(probabilities)
    if idx % 1000 == 0:
        print("Finished predicting", idx)


('Finished predicting', 0)
('Finished predicting', 1000)
('Finished predicting', 2000)
('Finished predicting', 3000)
('Finished predicting', 4000)
('Finished predicting', 5000)
('Finished predicting', 6000)
('Finished predicting', 7000)
('Finished predicting', 8000)
('Finished predicting', 9000)
('Finished predicting', 10000)
('Finished predicting', 11000)
('Finished predicting', 12000)
('Finished predicting', 13000)


In [19]:
#Compute utility for xgboost 
labels = np.concatenate(cohort_labels)
predictions = np.concatenate(cohort_predictions)
probabilities = np.concatenate(cohort_probabilities)

auroc, auprc = compute_auc(labels, probabilities)
accuracy, f_measure = compute_accuracy_f_measure(labels, predictions)

print("AUCROC, AUCPRC")
print(auroc, auprc)
print("Accuaracy, F-measure")
print(accuracy, f_measure) 

num_files = len(y_imputed_test)
dt_early = -12
dt_optimal = -6
dt_late = 3

max_u_tp = 1
min_u_fn = -2
u_fp = -0.05
u_tn = 0
# Compute utility.
observed_utilities = np.zeros(num_files)
best_utilities = np.zeros(num_files)
worst_utilities = np.zeros(num_files)
inaction_utilities = np.zeros(num_files)

for k in range(num_files):
    labels = cohort_labels[k]
    num_records = len(labels)
    observed_predictions = cohort_predictions[k]
    best_predictions = np.zeros(num_records)
    worst_predictions = np.zeros(num_records)
    inaction_predictions = np.zeros(num_records)

    if any(labels):
        t_sepsis = min(i for i, label in enumerate(
            labels) if label) - dt_optimal
        best_predictions[max(0, t_sepsis + dt_early)
                             : min(t_sepsis + dt_late, num_records)] = 1
    else:
        best_predictions[:] = 0
    worst_predictions = 1 - best_predictions

    observed_utilities[k] = compute_prediction_utility(
        labels, observed_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)
    best_utilities[k] = compute_prediction_utility(
        labels, best_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)
    worst_utilities[k] = compute_prediction_utility(
        labels, worst_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)
    inaction_utilities[k] = compute_prediction_utility(
        labels, inaction_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)

unnormalized_observed_utility = np.sum(observed_utilities)
unnormalized_best_utility = np.sum(best_utilities)
unnormalized_worst_utility = np.sum(worst_utilities)
unnormalized_inaction_utility = np.sum(inaction_utilities)

if not (unnormalized_worst_utility <= unnormalized_best_utility and unnormalized_inaction_utility <= unnormalized_best_utility):
    raise Exception(
        'Optimal utility must be higher than inaction utility.')

normalized_observed_utility = (unnormalized_observed_utility - unnormalized_inaction_utility) / (
    unnormalized_best_utility - unnormalized_inaction_utility)

print("Utility, ", normalized_observed_utility)

AUCROC, AUCPRC
(0.8192279586377739, 0.09921220055488239)
Accuaracy, F-measure
(0.9054936065366225, 0.15655257531246197)
('Utility, ', 0.3983772867477102)


In [20]:
#Save xgboost
from sklearn.externals import joblib
xgb_filepath = 'best_model_xg_boost_FT'
joblib.dump(xgb, xgb_filepath)

['best_model_xg_boost_FT']

In [21]:
#save scaler
scaler_file_path = 'scaler'
joblib.dump(scaler, scaler_file_path)

['scaler']

In [22]:
mean_feature_file_path = 'mean_features'
joblib.dump(mean_features, mean_feature_file_path)

['mean_features']

In [23]:
#Combine xgboost, rf, and dl
#Predict test data
autoencoder.load_weights(filepath)
#transformed_features = encoder.predict(normalized_train_features)

# test autoencoder + nr classifier model with test data
cohort_labels = []
cohort_predictions = []
cohort_probabilities = []
for idx, data in enumerate(X_imputed_test):
    num_row = data.shape[0]
    num_col = data.shape[1]
    x_input = []
    labels = []
    for i in range(num_row):
        if i >=5:
            d = data.loc[(i-5):i].values
        else:
            imputing_values = np.array(mean_features.values[0:40]).reshape(1, 40)
            d = data.loc[0:i].values
            
            while(d.shape[0] < 6):
                d = np.concatenate([imputing_values, d], axis=0)
            
        d = d.reshape((6, 40)).flatten()
        l = int(y_imputed_test[idx].loc[i])
        labels.append(l)
        x_input.append(d)
        
        
    x_input = np.array(x_input)
    
    nr_output = classifier.predict(scaler.transform(x_input).reshape((len(x_input), 6, 40)))
    nr_probabilities = nr_output[:,1]
    
    
    xgb_output = xgb.predict_proba(scaler.transform(x_input))
    xgb_probabilities = xgb_output[:,1]
    
    
    rf_output = clf.predict_proba(scaler.transform(x_input))
    rf_probabilities = rf_output[:,1]
    
    probabilities = []
    predictions = []
    for nr_p, xgb_p, rf_p in zip(nr_probabilities, xgb_probabilities, rf_probabilities):
        probabilities.append(np.max([nr_p,xgb_p,rf_p]))
        if (nr_p > 0.5 or xgb_p > 0.5 or rf_p > 0.5):
            predictions.append(1)
        else: predictions.append(0)
            
    predictions = np.array(predictions)
    
    cohort_labels.append(labels)
    cohort_predictions.append(predictions)
    cohort_probabilities.append(probabilities)
    if idx % 1000 == 0:
        print("Finished predicting", idx)


('Finished predicting', 0)
('Finished predicting', 1000)
('Finished predicting', 2000)
('Finished predicting', 3000)
('Finished predicting', 4000)
('Finished predicting', 5000)
('Finished predicting', 6000)
('Finished predicting', 7000)
('Finished predicting', 8000)
('Finished predicting', 9000)
('Finished predicting', 10000)
('Finished predicting', 11000)
('Finished predicting', 12000)
('Finished predicting', 13000)


In [24]:

labels = np.concatenate(cohort_labels)
predictions = np.concatenate(cohort_predictions)
probabilities = np.concatenate(cohort_probabilities)

#f = open('/home/manhnh594/predictions413.txt', 'w')
#for i in predictions:
#    f.write('%d \n' %i)
#f.close()
#print("Created file successfully")

auroc, auprc = compute_auc(labels, probabilities)
accuracy, f_measure = compute_accuracy_f_measure(labels, predictions)

print("AUCROC, AUCPRC")
print(auroc, auprc)
print("Accuaracy, F-measure")
print(accuracy, f_measure) 

num_files = len(y_imputed_test)
dt_early = -12
dt_optimal = -6
dt_late = 3

max_u_tp = 1
min_u_fn = -2
u_fp = -0.05
u_tn = 0
# Compute utility.
observed_utilities = np.zeros(num_files)
best_utilities = np.zeros(num_files)
worst_utilities = np.zeros(num_files)
inaction_utilities = np.zeros(num_files)

for k in range(num_files):
    labels = cohort_labels[k]
    num_records = len(labels)
    observed_predictions = cohort_predictions[k]
    best_predictions = np.zeros(num_records)
    worst_predictions = np.zeros(num_records)
    inaction_predictions = np.zeros(num_records)

    if any(labels):
        t_sepsis = min(i for i, label in enumerate(
            labels) if label) - dt_optimal
        best_predictions[max(0, t_sepsis + dt_early)
                             : min(t_sepsis + dt_late, num_records)] = 1
    else:
        best_predictions[:] = 0
    worst_predictions = 1 - best_predictions

    observed_utilities[k] = compute_prediction_utility(
        labels, observed_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)
    best_utilities[k] = compute_prediction_utility(
        labels, best_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)
    worst_utilities[k] = compute_prediction_utility(
        labels, worst_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)
    inaction_utilities[k] = compute_prediction_utility(
        labels, inaction_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)

unnormalized_observed_utility = np.sum(observed_utilities)
unnormalized_best_utility = np.sum(best_utilities)
unnormalized_worst_utility = np.sum(worst_utilities)
unnormalized_inaction_utility = np.sum(inaction_utilities)

if not (unnormalized_worst_utility <= unnormalized_best_utility and unnormalized_inaction_utility <= unnormalized_best_utility):
    raise Exception(
        'Optimal utility must be higher than inaction utility.')

normalized_observed_utility = (unnormalized_observed_utility - unnormalized_inaction_utility) / (
    unnormalized_best_utility - unnormalized_inaction_utility)

print("Utility, ", normalized_observed_utility)

AUCROC, AUCPRC
(0.8174128524208188, 0.09779186006432865)
Accuaracy, F-measure
(0.861254540673724, 0.12870003424825088)
('Utility, ', 0.41137950348287744)
